Решение состоит из трех этапов:
- предобработка текста
- сбор статистики
- генерация текста

In [ ]:
import pickle
import sys
from collections import defaultdict, Counter
import glob
import os
import getpass
import re
import string
import random



Предобработка текста:
это делается в классе  PrepareWords.
В результате предобработки в поле будет лежать лист из слов.
При этом точка, восклицательный знак, вопросительный знак, многоточие обрабатываются как отдельные слова. 

In [ ]:
class PrepareWords:
    def __init__(self):
        self.all_words = list()
        
    def separate_words(self, line):
        return re.findall(r"\w+|\!|\?|\.\.\.|\.", line)
       
    def prepare_text(self, text_path):
        with open(text_path) as text_file:
            for line in text_file:
                for word in self.separate_words(line):
                    self.all_words.append(word) 

Сбор статистики: 
это делается в классе GetStatistics.
На вход подается лист слов, сгенерированный на предыдущем этапе. 

Дальше идея такая: 
для реализации марковских цепей, надо уметь выдавать слово: 
а) по точке, восклицательному и вопросительному знакам, многоточию
(тогда за ними всегда будет слово с большой буквы).
б) по одному ключу
в) по двум ключам

Для этого будем использовать коллекции defaultdict и counter для получения статистики. Ключами будут tuple(words_buffer), где words_buffer - 
это маленький лист, в который будут поступать элементы и удаляться.
И он будет очищаться в случае, если встретили точку, какой-то знак, и т.д..

Тогда получится, что дефолтному пустому ключу tuple() (когда его очистили) соответствует начало нового предложения, значит, автоматически сгенерируется слово с большой буквы. 

Поддерживать точку никак дополнительно не придется, т.к. у нее будет своя частота появления. 
И слова, которые всегда начинаются с большой буквы, вроде местоемений, 
тоже никак отдельно обрабатывать не придется.


In [ ]:
class GetStatistics:
    def __init__(self, all_words):
        self.all_words = all_words
        self.words_buffer = list() 
        self.markov_chain = defaultdict(Counter)
        
    def append_word(self, word):
        buffer_length = len(self.words_buffer)
        if buffer_length == 0:
            self.words_buffer.append(word)
            key = tuple()
            self.markov_chain[key][word] += 1
            return
        
        if buffer_length == 1:
            self.words_buffer.append(word)
            key = tuple(self.words_buffer[0])
            self.markov_chain[key][self.words_buffer[1]] += 1
            return

        if buffer_length == 2:
            self.words_buffer.append(word)
            
        else:
            self.words_buffer.pop(0)
            self.words_buffer.append(word)

        key_of_one = tuple([self.words_buffer[1]])
        self.markov_chain[key_of_one][self.words_buffer[2]] += 1

        key_of_two = tuple([self.words_buffer[0], self.words_buffer[1]])
        self.markov_chain[key_of_two][self.words_buffer[2]] += 1
        
        
    def get_statistics(self, all_words):
        for word in all_words:
            self.append_word(word)
            if word in [".", "!", "...", "?"]:
                self.words_buffer = list()
        return self

Генерация текста: 
это делается в классе TextGenerator.

На вход подается количество параграфов. Количество предложений в параграфе = 
случайное число от ... до ...

Далее генерируется предложение:

Выдавать наиболее часто встречающееся слово будем методом most_common(n) из коллекции Counter.

In [ ]:
class TextGenerator:
    def __init__(self, markov_chain):
        self.markov_chain = markov_chain
        
    def gen_by_one(self, word):
        key = tuple([word])
        if key not in self.markov_chain.keys():
            return random.choice(self.markov_chain.keys())[0]
        return self.markov_chain[key].most_common(1)[0][0]

    def gen_by_two(self, first_word, second_word):
        key = tuple([first_word, second_word])
        if key not in self.markov_chain.keys():
            return random.choice(self.markov_chain.keys())[0]
        return self.markov_chain[key].most_common(1)[0][0]

    def gen_sentence(self):
        sentence = list()
        end_sentence = ""

        words_count = 1
        
        first_words = self.markov_chain[tuple()]
        first_word = random.choice(first_words.most_common(1))[0]
        sentence.append(first_word)

        second_word = self.gen_by_one(first_word)
        if second_word in [".", "!", "...", "?"]:
            end_sentence = second_word
            new_sentence = " ".join(sentence)
            return "".join([new_sentence, end_sentence])
        else:
            sentence.append(second_word)
            

        words_count += 1
        while not words_count >= 15 or second_word in [".", "!", "...", "?"]:
            new_word = self.gen_by_two(first_word, second_word)
            if new_word in [".", "!", "...", "?"]:
                end_sentence = new_word
                break
            sentence.append(new_word)
            first_word, second_word = second_word, new_word
            words_count += 1
            if words_count >= 15:
                end_sentence = "."
                break

        new_sentence = " ".join(sentence)
        return "".join([new_sentence, end_sentence])

    def gen_paragraph(self):
        paragraph = list()
        sentence_count = random.randint(1, 20)
        for i in range(sentence_count):
            paragraph.append(self.gen_sentence())
        paragraph.append("\n")
        return " ".join(paragraph)

    def gen_text(self, par_count):
        text = list()
        for i in range(par_count):
            text.append(self.gen_paragraph())
        return "".join(text)

Приложение:

скрипты, которые потребовались для сбора данных из корпуса, подгрузки и т.д.

Загрузили все в один txt:

In [ ]:
os.chdir("...")
read_files = glob.glob("*.txt")
path = "..."
with open(path, "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

Предобработка текста

In [ ]:
read_path = "..."
prepare_words = PrepareWords()
prepare_words.prepare_text(read_path)

Записали статистику с помощью pickle

In [ ]:
statistics = GetStatistics(prepare_words.all_words)
stat = statistics.get_statistics(prepare_words.all_words)

pickle_path = "..."
with open(pickle_path, "wb") as pickle_file:
    pickle.dump(stat, pickle_file)

Считали статистику с помощью pickle

In [ ]:
with open(pickle_path, "rb") as pickle_file:
    statistics = pickle.load(pickle_file)

Сгенерировали текст

In [ ]:
new_text_path = "..."
text_generator = TextGenerator(statistics.markov_chain)
with open(new_text_path, "w") as output_file:
    output_file.write(text_generator.gen_text(100))